## 1. Baseline code provided by uni

### 1.1 Import modules

In [1]:
import pandas as pd
import logging
import json
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import make_pipeline
from sklearn.dummy import DummyRegressor
from sklearn.linear_model import Ridge
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error

### 1.2 Baseline function to create [predictions](https://github.com/larshanen/MLChallenge/tree/main/notebooks/predicted.json)

In [2]:
'''
def main():
    # Set the logging level to INFO and set loading message
    logging.getLogger().setLevel(logging.INFO)
    
    # Load train and test sets and change all NA values to empty values
    logging.info("Loading training/test data")
    train = pd.DataFrame.from_records(json.load(open('../data/train.json'))).fillna("")
    test = pd.DataFrame.from_records(json.load(open('../data/test.json'))).fillna("")
    
    # Split the train set into train (75%) and validation (25%) sets
    logging.info("Splitting validation")
    train, val = train_test_split(train, stratify=train['year'], random_state=123)
    
    # Store a featurizer to transform the 'title' column into a bag-of-words format
    featurizer = ColumnTransformer(
        transformers=[("title", CountVectorizer(), "title")], remainder='drop')
    
    # Make a pipeline for the featurizer combined with a dummy regressor, that simply predicts the overall trained mean of the target variable
    dummy = make_pipeline(featurizer, DummyRegressor(strategy='mean'))

    # Make a pipeline for the featurizer and a ridge model, that aims to minimize the sum of squares
    ridge = make_pipeline(featurizer, Ridge())
    
    # Drop target variable column and fit both models
    logging.info("Fitting models")
    dummy.fit(train.drop('year', axis=1), train['year'].values)
    ridge.fit(train.drop('year', axis=1), train['year'].values)
    
    # Calculate and report both MAE's
    logging.info("Evaluating on validation data")
    err = mean_absolute_error(val['year'].values, dummy.predict(val.drop('year', axis=1)))
    logging.info(f"Mean baseline MAE: {err}")
    err = mean_absolute_error(val['year'].values, ridge.predict(val.drop('year', axis=1)))
    logging.info(f"Ridge regress MAE: {err}")
    
    # Let the ridge model predict on test set
    logging.info(f"Predicting on test")
    pred = ridge.predict(test)
    test['year'] = pred
    
    # Write JSON prediction file
    logging.info("Writing prediction file")
    test.to_json("predicted.json", orient='records', indent=2)
'''

'\ndef main():\n    # Set the logging level to INFO and set loading message\n    logging.getLogger().setLevel(logging.INFO)\n    \n    # Load train and test sets and change all NA values to empty values\n    logging.info("Loading training/test data")\n    train = pd.DataFrame.from_records(json.load(open(\'../data/train.json\'))).fillna("")\n    test = pd.DataFrame.from_records(json.load(open(\'../data/test.json\'))).fillna("")\n    \n    # Split the train set into train (75%) and validation (25%) sets\n    logging.info("Splitting validation")\n    train, val = train_test_split(train, stratify=train[\'year\'], random_state=123)\n    \n    # Store a featurizer to transform the \'title\' column into a bag-of-words format\n    featurizer = ColumnTransformer(\n        transformers=[("title", CountVectorizer(), "title")], remainder=\'drop\')\n    \n    # Make a pipeline for the featurizer combined with a dummy regressor, that simply predicts the overall trained mean of the target variable\n 

In [3]:
# main()

## 2. Team code

Please follow the instructions beneath when writing or adjusting code:

In [4]:
# Describe every piece of code with comments
# Include your name in every header so we can report our individual contributions (this is mandatory)

### 2.1 Explore baseline performance (Lars)

In [5]:
# Set the logging level to INFO and set loading message
logging.getLogger().setLevel(logging.INFO)
    
# Load train and test sets and change all NA values to empty values
logging.info("Loading training/test data")
train = pd.DataFrame.from_records(json.load(open('../../data/train.json')))
test = pd.DataFrame.from_records(json.load(open('../../data/test.json')))

INFO:root:Loading training/test data


In [6]:
"""
# Split the train set into train (75%) and validation (25%) sets
logging.info("Splitting validation")
train, val = train_test_split(train, stratify=train['year'], random_state=123)
    
# Store a featurizer to transform the 'title' column into a bag-of-words format
featurizer = ColumnTransformer(
transformers=[("title", CountVectorizer(), "title")], remainder='drop')
    
# Make a pipeline for the featurizer combined with a dummy regressor, that simply predicts the overall trained mean of the target variable
dummy = make_pipeline(featurizer, DummyRegressor(strategy='mean'))

# Make a pipeline for the featurizer and a ridge model, that aims to minimize the sum of squares
ridge = make_pipeline(featurizer, Ridge())
    
# Drop target variable column and fit both models
logging.info("Fitting models")
dummy.fit(train.drop('year', axis=1), train['year'].values)
ridge.fit(train.drop('year', axis=1), train['year'].values)
    
# Calculate and report both MAE's
logging.info("Evaluating on validation data")
err = mean_absolute_error(val['year'].values, dummy.predict(val.drop('year', axis=1)))
logging.info(f"Mean baseline MAE: {err}")
err = mean_absolute_error(val['year'].values, ridge.predict(val.drop('year', axis=1)))
logging.info(f"Ridge regress MAE: {err}")
"""

'\n# Split the train set into train (75%) and validation (25%) sets\nlogging.info("Splitting validation")\ntrain, val = train_test_split(train, stratify=train[\'year\'], random_state=123)\n    \n# Store a featurizer to transform the \'title\' column into a bag-of-words format\nfeaturizer = ColumnTransformer(\ntransformers=[("title", CountVectorizer(), "title")], remainder=\'drop\')\n    \n# Make a pipeline for the featurizer combined with a dummy regressor, that simply predicts the overall trained mean of the target variable\ndummy = make_pipeline(featurizer, DummyRegressor(strategy=\'mean\'))\n\n# Make a pipeline for the featurizer and a ridge model, that aims to minimize the sum of squares\nridge = make_pipeline(featurizer, Ridge())\n    \n# Drop target variable column and fit both models\nlogging.info("Fitting models")\ndummy.fit(train.drop(\'year\', axis=1), train[\'year\'].values)\nridge.fit(train.drop(\'year\', axis=1), train[\'year\'].values)\n    \n# Calculate and report both M

### 2.2 Preprocessing (Lars)

This paragraph build upon the previous baseline code. It entails the following adjustments/additions chronologically:

- [x] Removal of dummy regressor, since ridge works better from the very start;
- [x] 5-fold cross validation to reduce variability (Ridge regress MAE (5.773));
- [x] Entrytype: Dummy-encoded;
- [x] Title: TF-IDF vectorized, limited to top X (hyperparameter) occuring (non-stopword) words;
- [x] Editor: Removed, >75% missing values;
- [x] Publisher: Count vectorized, limited to top X (hyperparameter) occuring publishers;
- [x] Author: Count vectorized, limited to top X (hyperparameter) occuring authors;
- [x] Abstract: TF-IDF vectorized, limited to top X (hyperparameter) occuring (non-stopword) words;
- [ ] Try tasks other than regression, like lazy learning (kNN)(?);
- [ ] Try BERTopic modelling;

In [7]:
# Import extra modules
from sklearn.model_selection import cross_val_score, KFold
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import MultiLabelBinarizer
import numpy as np

For experimental purposes we start working with a 10% subset of the data:

In [8]:
#Randomly save 10% of the train set for velocity purposes
percentage_to_save = 30

# Calculate the number of rows to save
num_rows_to_save = int(len(train) * (percentage_to_save / 100))

# Use the sample method to randomly select rows
train_sample = train.sample(n=num_rows_to_save, random_state=42)  # Set a random_state for reproducibility

train_sample.head()

,ENTRYTYPE,title,editor,year,publisher,author,abstract
12680,inproceedings,Question-Answering Based on Virtually Integrat...,None,2003,Association for Computational Linguistics,"[Choi, Key-Sun, Kim, Jae-Ho, Miyazaki, Masaru,...",None
17292,inproceedings,AMI&ERIC: How to Learn with Naive Bayes and Pr...,None,2013,Association for Computational Linguistics,"[Dermouche, Mohamed, Khouas, Leila, Velcin, Ju...",None
33265,inproceedings,Inducing Gazetteers for Named Entity Recogniti...,None,2008,Association for Computational Linguistics,"[Kazama, Jun'ichi, Torisawa, Kentaro]",None
52850,inproceedings,Leveraging Explicit Lexico-logical Alignments ...,None,2022,Association for Computational Linguistics,"[Sun, Runxin, He, Shizhu, Zhu, Chong, He, Yaoh...",Text-to-SQL aims to parse natural language que...
2298,inproceedings,CLAM: Quickly deploy NLP command-line tools on...,None,2014,Dublin City University and Association for Com...,"[van Gompel, Maarten, Reynaert, Martin]",None


#### 2.2.1 Drop all columns with over 75% of missing data

In [9]:
# Set threshold on 75%
threshold = 0.25

# Calculate the threshold for each column
missing_threshold = int(threshold * len(train_sample))

# Drop columns with more than the specified percentage of missing data
train_filtered = train_sample.dropna(axis=1, thresh=missing_threshold)

print(len(train_filtered))
train_filtered.head()

19774


,ENTRYTYPE,title,year,publisher,author,abstract
12680,inproceedings,Question-Answering Based on Virtually Integrat...,2003,Association for Computational Linguistics,"[Choi, Key-Sun, Kim, Jae-Ho, Miyazaki, Masaru,...",None
17292,inproceedings,AMI&ERIC: How to Learn with Naive Bayes and Pr...,2013,Association for Computational Linguistics,"[Dermouche, Mohamed, Khouas, Leila, Velcin, Ju...",None
33265,inproceedings,Inducing Gazetteers for Named Entity Recogniti...,2008,Association for Computational Linguistics,"[Kazama, Jun'ichi, Torisawa, Kentaro]",None
52850,inproceedings,Leveraging Explicit Lexico-logical Alignments ...,2022,Association for Computational Linguistics,"[Sun, Runxin, He, Shizhu, Zhu, Chong, He, Yaoh...",Text-to-SQL aims to parse natural language que...
2298,inproceedings,CLAM: Quickly deploy NLP command-line tools on...,2014,Dublin City University and Association for Com...,"[van Gompel, Maarten, Reynaert, Martin]",None


#### 2.2.2 Featurize 'author' column (count-vectors, reduced to top X most frequent authors)

In [10]:
# Convert lists of strings, accounting for None values
train_filtered['author_str'] = train_filtered['author'].apply(lambda x: ';'.join(map(str, x)) if x is not None else 'unknown')

# Add a column to store the original row numbers
# train_filtered['original_index'] = train_filtered.index

# Count the number of papers for each author
author_paper_counts = train_filtered['author_str'].str.split(';').explode().value_counts()

# Set the number of most frequent authors you want to include
n_mostfreq_authors = 120  # Adjust this value to the desired number of most frequent authors

# Filter authors based on the X most frequent authors
top_authors = author_paper_counts.head(n_mostfreq_authors).index.tolist()

# Filter only the top authors in 'author_str'
train_filtered['author_str_filtered'] = train_filtered['author_str'].apply(lambda x: ';'.join([author for author in x.split(';') if author in top_authors]))

# Count-vectorize 'author_str_filtered'
count_vectorizer = CountVectorizer(tokenizer=lambda x: x.split(';'))
count_matrix = count_vectorizer.fit_transform(train_filtered['author_str_filtered'])

# Extract and create columns
feature_names = count_vectorizer.get_feature_names_out()
author_count_df = pd.DataFrame(count_matrix.toarray(), columns=feature_names)

# Set the original_index column as the index
# author_count_df.set_index(train_filtered['original_index'], inplace=True)

author_count_df = author_count_df.drop(['unknown', ''], axis=1) # See if this approach always works out

print(len(author_count_df))
author_count_df.head()

19774


c:\Users\Gebruiker\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\feature_extraction\text.py:525: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


,"baldwin, timothy","bandyopadhyay, sivaji","bansal, mohit","baroni, marco","bhattacharyya, pushpak","biemann, chris","bojar, ondřej","bond, francis","callison-burch, chris","cardie, claire",...,"zhang, min","zhang, qi","zhang, yue","zhao, hai","zhao, jun","zhou, guodong","zhou, jie","zhou, ming","zong, chengqing","øvrelid, lilja"
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


Merge with train_filtered, meaning we drop the author column and then add author_count_df

In [11]:
# Drop all redundant columns
# train_filtered_buffer = train_filtered.drop(['author', 'author_str', 'author_str_filtered'], axis=1)

# Concatenate the original with dropped redundants and the extracted features for author
# train_2 = pd.concat([train_filtered_buffer, author_count_df], axis=1).reindex(train_filtered_buffer.index)

# print(len(train_2))
# train_2.head()

#### 2.2.3 Vectorize 'ENTRYTYPE' column (3-categorical variable one-hot encoded)

In [12]:
# Perform one-hot encoding
train_encoded_entrytype = pd.get_dummies(train_filtered['ENTRYTYPE'], columns=['category'], prefix='category')

# Show count-values for each of the columns
train_encoded_entrytype.apply(lambda x: x.value_counts())

,category_article,category_inproceedings,category_proceedings
False,19139,1177,19232
True,635,18597,542


Merge with train_filtered, meaning we drop the ENTRYTYPE column and then add train_encoded_entrytype

In [13]:
# Drop all redundant columns
# train_2 = train_2.drop(['ENTRYTYPE'], axis=1)

# Concatenate the original with dropped redundants and the extracted features for ENTRYTYPE
# train_3 = pd.concat([train_2, train_encoded_entrytype], axis=1).reindex(train_2.index)

# print(len(train_3))
# train_3.head()

#### 2.2.4 Vectorize 'Publisher' column (116-categorical variable count-vectorized, and reduced to X most frequent publishers)

In [14]:
# Set the number of most frequent publishers to include
n_mostfreq_publishers = 20  # Adjust this value as needed

# Get the X most frequent publishers
top_publishers = train_filtered['publisher'].value_counts().head(n_mostfreq_publishers).index.tolist()

# Create a new DataFrame with one-hot encoding for the X most frequent publishers
train_encoded_publisher = pd.get_dummies(train_filtered['publisher'][train_filtered['publisher'].isin(top_publishers)], prefix='publisher')

# Show count-values for each of the columns
train_encoded_publisher.apply(lambda x: x.value_counts())

,publisher_ATALA,publisher_ATALA (Association pour le Traitement Automatique des Langues),publisher_Asian Federation of Natural Language Processing,publisher_Aslib,publisher_Association for Computational Linguistics,publisher_Association for Machine Translation in the Americas,publisher_COLING,publisher_Coling 2008 Organizing Committee,publisher_Coling 2010 Organizing Committee,publisher_European Association for Machine Translation,publisher_European Language Resources Association,publisher_European Language Resources Association (ELRA),publisher_INCOMA Ltd.,"publisher_INCOMA Ltd. Shoumen, BULGARIA",publisher_International Committee on Computational Linguistics,publisher_MIT Press,publisher_NLP Association of India,publisher_The Association for Computational Linguistics and Chinese Language Processing (ACLCLP),publisher_The COLING 2012 Organizing Committee,publisher_The COLING 2016 Organizing Committee
False,15526,15839,15710,15776,4748,15726,15804,15826,15812,15710,15357,14578,15778,15839,15603,15524,15844,15674,15787,15734
True,379,66,195,129,11157,179,101,79,93,195,548,1327,127,66,302,381,61,231,118,171


Merge with train_filtered, meaning we drop the 'publisher' column and then add train_encoded_entrytype

In [15]:
# Drop all redundant columns
# train_3 = train_3.drop(['publisher'], axis=1)

# Concatenate the original with dropped redundants and the extracted features for ENTRYTYPE
# train_4 = pd.concat([train_3, train_encoded_publisher], axis=1).reindex(train_3.index).fillna(0)

# print(len(train_4))
# train_4.head()

#### 2.2.5.1 Vectorize 'title' and 'abstract' column (English-translated with stop-words removal and/or synonym replacement)

In [16]:
'''
from googletrans import Translator
from langdetect import detect
'''

'\nfrom googletrans import Translator\nfrom langdetect import detect\n'

In [17]:
'''
def det(x):
    try:
        lang = detect(x)
    except:
        lang = 'Other'
    return lang
'''

"\ndef det(x):\n    try:\n        lang = detect(x)\n    except:\n        lang = 'Other'\n    return lang\n"

In [18]:
'''
train_filtered['language_title'] = train_filtered['title'].apply(det)
train_filtered['language_abstract'] = train_filtered['abstract'].apply(det)
'''

"\ntrain_filtered['language_title'] = train_filtered['title'].apply(det)\ntrain_filtered['language_abstract'] = train_filtered['abstract'].apply(det)\n"

In [19]:
'''
translator = Translator(service_urls=['translate.googleapis.com'])

# Function to translate non-English titles to English based on 'translated_title' column
def translate_to_english(dataframe, column, translated_column):
    for i in dataframe[column].index:
        # Check if the value in 'translated_title' is not 'en' or 'Other' before translation
        if dataframe[translated_column][i] not in ['en', 'Other']:
            dataframe[column][i] = translator.translate(dataframe[column][i], dest='en').text
'''

"\ntranslator = Translator(service_urls=['translate.googleapis.com'])\n\n# Function to translate non-English titles to English based on 'translated_title' column\ndef translate_to_english(dataframe, column, translated_column):\n    for i in dataframe[column].index:\n        # Check if the value in 'translated_title' is not 'en' or 'Other' before translation\n        if dataframe[translated_column][i] not in ['en', 'Other']:\n            dataframe[column][i] = translator.translate(dataframe[column][i], dest='en').text\n"

In [20]:
'''
from deep_translator import GoogleTranslator

# Function to translate non-English titles to English based on 'translated_title' column
def translate_to_english(dataframe, column, translated_column):
    for i in dataframe[column].index:
        # Check if the value in 'translated_title' is not 'en' or 'Other' before translation
        if dataframe[translated_column][i] not in ['en', 'Other']:
            dataframe[column][i] = GoogleTranslator(source='auto', target='en').translate(text=dataframe[column][i])
'''

"\nfrom deep_translator import GoogleTranslator\n\n# Function to translate non-English titles to English based on 'translated_title' column\ndef translate_to_english(dataframe, column, translated_column):\n    for i in dataframe[column].index:\n        # Check if the value in 'translated_title' is not 'en' or 'Other' before translation\n        if dataframe[translated_column][i] not in ['en', 'Other']:\n            dataframe[column][i] = GoogleTranslator(source='auto', target='en').translate(text=dataframe[column][i])\n"

In [21]:
'''
translate_to_english(train_filtered, 'title', 'language_title')
translate_to_english(train_filtered, 'abstract', 'language_abstract')
'''

"\ntranslate_to_english(train_filtered, 'title', 'language_title')\ntranslate_to_english(train_filtered, 'abstract', 'language_abstract')\n"

In [22]:
# train_4['new_language_title'] = train_4['title'].apply(det)
# train_4['new_language_abstract'] = train_4['abstract'].apply(det)

In [23]:
# train_4['new_language_title'].value_counts()

In [24]:
# train_4['new_language_abstract'].value_counts()

In [25]:
train_textcolumns = train_filtered[['title', 'abstract']]
train_textcolumns

,title,abstract
12680,Question-Answering Based on Virtually Integrat...,None
17292,AMI&ERIC: How to Learn with Naive Bayes and Pr...,None
33265,Inducing Gazetteers for Named Entity Recogniti...,None
52850,Leveraging Explicit Lexico-logical Alignments ...,Text-to-SQL aims to parse natural language que...
2298,CLAM: Quickly deploy NLP command-line tools on...,None
...,...,...
7809,Usability Recommendations for Annotation Tools,None
6236,Deep Span Representations for Named Entity Rec...,Span-based models are one of the most straight...
63616,Comparing a Hand-crafted to an Automatically G...,The automatic evaluation of machine translatio...
6396,Thai Stock News Sentiment Classification using...,None


In [26]:
# Specify the number of most frequent tokens you want to keep (replace X with the desired value)
max_features_title = 500

# Create a list of English stopwords
stop_words = 'english'

# Apply the TF-IDF vectorizer to column 'title' with max_features parameter
tfidf_vectorizer = TfidfVectorizer(stop_words=stop_words, max_features=max_features_title)

# Apply the TF-IDF vectorizer to column 'title'
tfidf_matrix_title = tfidf_vectorizer.fit_transform(train_textcolumns['title'])

# Extract and create columns
feature_names = tfidf_vectorizer.get_feature_names_out()
tfidf_df_title = pd.DataFrame(tfidf_matrix_title.toarray(), columns=feature_names)

print(f"We've transformed the 'title' column to a dataframe of {len(tfidf_df_title.columns)} columns.")
tfidf_df_title.head()

We've transformed the 'title' column to a dataframe of 500 columns.


,19,2016,2017,2018,2019,2020,2021,2022,2023,abstract,...,web,wikipedia,wmt,word,wordnet,words,workshop,world,writing,zero
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.545583,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [27]:
# Specify the number of most frequent tokens you want to keep (replace X with the desired value)
max_features_abstract = 500

# Create a list of English stopwords
stop_words = 'english'

# Handle missing values and '0's in the 'abstract' column
train_textcolumns['abstract'].replace({None: '', '0': ''}, inplace=True)

# Apply the TF-IDF vectorizer to column 'title' with max_features parameter
tfidf_vectorizer = TfidfVectorizer(stop_words=stop_words, max_features=max_features_abstract)

# Apply the TF-IDF vectorizer to column 'title'
tfidf_matrix_abstract = tfidf_vectorizer.fit_transform(train_textcolumns['abstract'])

# Extract and create columns
feature_names = tfidf_vectorizer.get_feature_names_out()
tfidf_df_abstract = pd.DataFrame(tfidf_matrix_abstract.toarray(), columns=feature_names)

print(f"We've transformed the 'abstract' column to a dataframe of {len(tfidf_df_abstract.columns)} columns.")
tfidf_df_abstract.head()

C:\Users\Gebruiker\AppData\Local\Temp\ipykernel_22932\261390455.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_textcolumns['abstract'].replace({None: '', '0': ''}, inplace=True)


We've transformed the 'abstract' column to a dataframe of 500 columns.


,ability,able,accuracy,achieve,achieved,achieves,adaptation,addition,additional,address,...,web,wikipedia,word,words,work,works,world,written,years,zero
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.173568,0.0,...,0.0,0.0,0.0,0.0,0.115918,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0


### 3. Merge the following dataframes: author_count_df, train_encoded_entrytype, train_encoded_publisher, tfidf_df_title, tfidf_df_abstract

In [28]:
year_df = train_sample['year']

features_df = pd.concat([author_count_df, train_encoded_entrytype, train_encoded_publisher, tfidf_df_title, tfidf_df_abstract, year_df], axis=1).reindex(year_df.index).fillna(0)

In [29]:
features_df

,"baldwin, timothy","bandyopadhyay, sivaji","bansal, mohit","baroni, marco","bhattacharyya, pushpak","biemann, chris","bojar, ondřej","bond, francis","callison-burch, chris","cardie, claire",...,wikipedia,word,words,work,works,world,written,years,zero,year
12680,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,2003
17292,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,2013
33265,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,2008
52850,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,2022
2298,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.062171,0.0,0.0,0.0,0.0,0.100883,2014
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7809,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,2012
6236,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,2023
63616,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,2019
6396,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,2015


### 4. Fit baseline model

In [ ]:
def train_model(df, m, name):
    model = m
    train, val = train_test_split(df, stratify=df['year'], random_state=123)
    model.fit(train.drop('year', axis=1), train['year'].values)
    predictions = model.predict(val.drop('year', axis=1))
    mae = mean_absolute_error(val['year'].values, predictions)
    print("{0} mae {1}".format(name,mae))
    

reg2 = RandomForestRegressor(random_state=3,n_estimators =100,max_depth=4)
reg1 = XGBRegressor(n_estimators=1000)
ereg = VotingRegressor(estimators=[('gb', reg1), ('rf', reg2)])
ereg = ereg.fit(x_train, y_train)



train_model(features_df, DecisionTreeRegressor(),"Decision Tree Regressor")
train_model(features_df, RandomForestRegressor(random_state=3),"Random Forest Regressor")   
train_model(features_df, XGBRegressor(n_estimators=600),"XGBoost Regressor")   
train_model(features_df, GradientBoostingRegressor(random_state=1),"Gradient Regressor")   
train_model(features_df, ereg,"Voting Regressor")
train_model(features_df, ExtraTreesRegressor(),"Extra Trees Regressor")   
train_model(features_df, LinearRegression(),"Linear Regressor")

In [30]:
# Split the train set into train (75%) and validation (25%) sets
logging.info("Splitting validation")
train, val = train_test_split(features_df, stratify=features_df['year'], random_state=123)

# Make a pipeline for the featurizer and a ridge model, that aims to minimize the sum of squares
ridge = Ridge()
    
# Drop target variable column and fit both models
logging.info("Fitting models")
ridge.fit(train.drop('year', axis=1), train['year'].values)
    
# Calculate and report both MAE's
logging.info("Evaluating on validation data")
err = mean_absolute_error(val['year'].values, ridge.predict(val.drop('year', axis=1)))
logging.info(f"Ridge regress MAE: {err}")

INFO:root:Splitting validation
INFO:root:Fitting models
INFO:root:Evaluating on validation data
INFO:root:Ridge regress MAE: 6.521680811338317


In [31]:
# Split the train set into train (75%) and validation (25%) sets
logging.info("Splitting validation")
train, val = train_test_split(features_df, stratify=features_df['year'], random_state=123)

# Make a pipeline for the featurizer and a ridge model, that aims to minimize the sum of squares
forest = RandomForestRegressor()
    
# Drop target variable column and fit both models
logging.info("Fitting models")
forest.fit(train.drop('year', axis=1), train['year'].values)
    
# Calculate and report both MAE's
logging.info("Evaluating on validation data")
err = mean_absolute_error(val['year'].values, ridge.predict(val.drop('year', axis=1)))
logging.info(f"Ridge regress MAE: {err}")

INFO:root:Splitting validation
INFO:root:Fitting models


KeyboardInterrupt: 

### 2.3 Incorporate into baseline code

In [ ]:
'''
# Set the logging level to INFO and set loading message
logging.getLogger().setLevel(logging.INFO)
    
# Load train and test sets and change all NA values to empty values
logging.info("Loading training/test data")
#train = pd.DataFrame.from_records(json.load(open('../data/train.json'))).fillna("")
#test = pd.DataFrame.from_records(json.load(open('../data/test.json'))).fillna("")
    
# Split the train set into train (80%) and validation (20%) sets, 5-folds
logging.info("Splitting validation")
num_folds = 5
k_fold = KFold(n_splits=num_folds, shuffle=True, random_state=123)
    
# Store a featurizer to transform the 'title' column into a bag-of-words format
featurizer_1 = ColumnTransformer(
    transformers=[("title", CountVectorizer(), "title")], remainder='drop')
featurizer_2 = ColumnTransformer(
    transformers=[("title", TfidfVectorizer(), "title")], remainder='drop')
featurizer_3 = ColumnTransformer(
    transformers=[("abstract", CountVectorizer(), "abstract")], remainder='drop')
featurizer_4 = ColumnTransformer(
    transformers=[("abstract", TfidfVectorizer(), "abstract")], remainder='drop')
featurizers = [featurizer_1, featurizer_2, featurizer_3, featurizer_4]

for i, featurizer in enumerate(featurizers):
    # Make a pipeline for the featurizer and a ridge model, that aims to minimize the sum of squares
    ridge_cv = make_pipeline(featurizer, Ridge())
    
    # Drop target variable column and fit both models
    logging.info(f"Fitting model with featurizer {i+1}")
    ridge_cv.fit(train_4.drop('year', axis=1), train_4['year'].values)
    
    # Calculate and report both MAE's
    logging.info("Evaluating on validation data")
    ridge_cv_scores = cross_val_score(ridge_cv, train_4.drop('year', axis=1), train_4['year'].values, cv=k_fold, scoring='neg_mean_absolute_error')
    logging.info(f"Ridge regress MAE with featurizer {i+1} ({num_folds}-fold cross-validated): {-ridge_cv_scores.mean()}")
'''

'\n# Set the logging level to INFO and set loading message\nlogging.getLogger().setLevel(logging.INFO)\n    \n# Load train and test sets and change all NA values to empty values\nlogging.info("Loading training/test data")\n#train = pd.DataFrame.from_records(json.load(open(\'../data/train.json\'))).fillna("")\n#test = pd.DataFrame.from_records(json.load(open(\'../data/test.json\'))).fillna("")\n    \n# Split the train set into train (80%) and validation (20%) sets, 5-folds\nlogging.info("Splitting validation")\nnum_folds = 5\nk_fold = KFold(n_splits=num_folds, shuffle=True, random_state=123)\n    \n# Store a featurizer to transform the \'title\' column into a bag-of-words format\nfeaturizer_1 = ColumnTransformer(\n    transformers=[("title", CountVectorizer(), "title")], remainder=\'drop\')\nfeaturizer_2 = ColumnTransformer(\n    transformers=[("title", TfidfVectorizer(), "title")], remainder=\'drop\')\nfeaturizer_3 = ColumnTransformer(\n    transformers=[("abstract", CountVectorizer(),